https://edumunozsala.github.io/BlogEms/jupyter/nlp/classification/embeddings/python/2020/08/15/Intro_NLP_WordEmbeddings_Classification.html#Large-Movie-Review-Dataset

https://github.com/corazzon/KaggleStruggle/blob/master/word2vec-nlp-tutorial/tutorial-part-1.ipynb

제출 : https://www.kaggle.com/competitions/word2vec-nlp-tutorial/submissions
  - 83.3 %

# 1. 영어 Word2Vec 만들기

라이브러리 로드

In [1]:
import pandas as pd
import numpy as np
# import os
import matplotlib.pyplot as plt
import seaborn as sn
# import pickle

%matplotlib inline

#Import module to split the datasets
# from sklearn.model_selection import train_test_split
# Import modules to evaluate the metrics
# from sklearn import metrics
# from sklearn.metrics import confusion_matrix,accuracy_score,roc_auc_score,roc_curve,auc

## 1) 데이터 이해

In [2]:
"""
header = 0 은 파일의 첫 번째 줄에 열 이름이 있음을 나타내며
delimiter = \t 는 필드가 탭으로 구분되는 것을 의미한다.
quoting = 3은 쌍따옴표를 무시하도록 한다.
"""
# QUOTE_MINIMAL (0), QUOTE_ALL (1),
# QUOTE_NONNUMERIC (2) or QUOTE_NONE (3).

# 레이블인 sentiment 가 있는 학습 데이터
train_path = pd.read_csv('https://github.com/corazzon/KaggleStruggle/raw/master/word2vec-nlp-tutorial/data/labeledTrainData.tsv', delimiter='\t', quoting=3)
# # 레이블이 없는 테스트 데이터
test_path = pd.read_csv('https://github.com/corazzon/KaggleStruggle/raw/master/word2vec-nlp-tutorial/data/testData.tsv', delimiter='\t', quoting=3)
train_path.shape

(25000, 3)

In [3]:
!pip show BeautifulSoup4

Name: beautifulsoup4
Version: 4.12.3
Summary: Screen-scraping library
Home-page: 
Author: 
Author-email: Leonard Richardson <leonardr@segfault.org>
License: MIT License
Location: /usr/local/lib/python3.10/dist-packages
Requires: soupsieve
Required-by: gdown, google, nbconvert, yfinance


In [4]:
from bs4 import BeautifulSoup
import re
import nltk
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('english')

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
from nltk.corpus import stopwords

In [7]:
def review_to_words( raw_review ):
    # 1. HTML 제거
    review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
    # 2. 영문자가 아닌 문자는 공백으로 변환
    letters_only = re.sub('[^a-zA-Z]', ' ', review_text)
    # 3. 소문자 변환
    words = letters_only.lower().split()
    # 4. 파이썬에서는 리스트보다 세트로 찾는게 훨씬 빠르다.
    # stopwords 를 세트로 변환한다.
    stops = set(stopwords.words('english'))
    # 5. Stopwords 불용어 제거
    meaningful_words = [w for w in words if not w in stops]
    # 6. 어간추출
    stemming_words = [stemmer.stem(w) for w in meaningful_words]
    # 7. 공백으로 구분된 문자열로 결합하여 결과를 반환
    return( ' '.join(stemming_words) )

In [8]:
from multiprocessing import Pool
import numpy as np

def _apply_df(args):
    df, func, kwargs = args
    return df.apply(func, **kwargs)

def apply_by_multiprocessing(df, func, **kwargs):
    # 키워드 항목 중 workers 파라메터를 꺼냄
    workers = kwargs.pop('workers')
    # 위에서 가져온 workers 수로 프로세스 풀을 정의
    pool = Pool(processes=workers)
    # 실행할 함수와 데이터프레임을 워커의 수 만큼 나눠 작업
    result = pool.map(_apply_df, [(d, func, kwargs)
            for d in np.array_split(df, workers)])
    pool.close()
    # 작업 결과를 합쳐서 반환
    return pd.concat(list(result))

In [39]:
%time X_train = apply_by_multiprocessing(\
    train_path['review'], review_to_words, workers=4)

<ipython-input-7-5a1938eb7345>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
<ipython-input-7-5a1938eb7345>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
<ipython-input-7-5a1938eb7345>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
<ipython-input-7-5a1938eb7345>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(ra

CPU times: user 159 ms, sys: 99.6 ms, total: 258 ms
Wall time: 32.7 s


In [40]:
%time X_test = apply_by_multiprocessing(\
    test_path['review'], review_to_words, workers=4)

<ipython-input-7-5a1938eb7345>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
<ipython-input-7-5a1938eb7345>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
<ipython-input-7-5a1938eb7345>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
<ipython-input-7-5a1938eb7345>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(ra

CPU times: user 138 ms, sys: 156 ms, total: 294 ms
Wall time: 28.6 s


In [11]:
Ytrain = train_path['sentiment']

In [12]:
# X_train = pd.DataFrame(X_train, columns=['review'])
# X_test = pd.DataFrame(X_test, columns=['review'])

In [41]:
# merged_df = pd.concat([train_path['review'], test_path['review']], axis=0)
merged_df = pd.concat([X_train, X_test], axis=0)

merged_df.shape

(50000,)

https://wikidocs.net/50739

In [42]:
# parse_text = '\n'.join(df['review'])
# parse_text = '\n'.join(merged_df)

In [43]:
import re
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
nltk.download('punkt')

# 문장 토큰화
# sent_text = sent_tokenize(parse_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [44]:
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
# normalized_text = []
# for string in sent_text:
#      tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
#      normalized_text.append(tokens)

In [49]:
# 단어 토큰화
result = [word_tokenize(sentence) for sentence in merged_df]
# result = [word_tokenize(sentence) for sentence in sent_text]

In [50]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 50000


In [51]:
# 샘플 3개만 출력
for line in result[:3]:
    print(line)

['stuff', 'go', 'moment', 'mj', 'start', 'listen', 'music', 'watch', 'odd', 'documentari', 'watch', 'wiz', 'watch', 'moonwalk', 'mayb', 'want', 'get', 'certain', 'insight', 'guy', 'thought', 'realli', 'cool', 'eighti', 'mayb', 'make', 'mind', 'whether', 'guilti', 'innoc', 'moonwalk', 'part', 'biographi', 'part', 'featur', 'film', 'rememb', 'go', 'see', 'cinema', 'origin', 'releas', 'subtl', 'messag', 'mj', 'feel', 'toward', 'press', 'also', 'obvious', 'messag', 'drug', 'bad', 'kay', 'visual', 'impress', 'cours', 'michael', 'jackson', 'unless', 'remot', 'like', 'mj', 'anyway', 'go', 'hate', 'find', 'bore', 'may', 'call', 'mj', 'egotist', 'consent', 'make', 'movi', 'mj', 'fan', 'would', 'say', 'made', 'fan', 'true', 'realli', 'nice', 'actual', 'featur', 'film', 'bit', 'final', 'start', 'minut', 'exclud', 'smooth', 'crimin', 'sequenc', 'joe', 'pesci', 'convinc', 'psychopath', 'power', 'drug', 'lord', 'want', 'mj', 'dead', 'bad', 'beyond', 'mj', 'overheard', 'plan', 'nah', 'joe', 'pesci', 

## 3) FastText 학습

https://wikidocs.net/22883

- FastText() : 기존 임베딩은 초기화되고 새로운 데이터를 통해 새로운 임베딩이 학습
- fasttext.train_supervised() : 기존 임베딩은 유지되고 새로운 데이터를 추가하여 임베딩이 업데이트

In [52]:
from gensim.models import FastText

#model = FastText(result, vector_size=100, window=5, min_count=5, workers=4, sg=1)
%time model = FastText(result, window=5, min_count=5, workers=4, sg=1)

CPU times: user 6min 21s, sys: 574 ms, total: 6min 21s
Wall time: 3min 25s


In [53]:
# Word2Vec는 학습하지 않은 단어에 대해서 유사한 단어를 찾아내지 못 했지만,
# FastText는 유사한 단어를 계산해서 출력하고 있음을 볼 수 있습니다.
model.wv.most_similar("electrofishing")

[('electrolyt', 0.8856043219566345),
 ('electro', 0.8509320020675659),
 ('electroshock', 0.8454113006591797),
 ('electra', 0.8417786955833435),
 ('electromagnet', 0.8348149657249451),
 ('electr', 0.808749258518219),
 ('electron', 0.802282989025116),
 ('electrician', 0.7980889678001404),
 ('electrocut', 0.7949947714805603),
 ('electrifi', 0.7823549509048462)]

In [54]:
# 모델 저장
model.save("fasttext_model.bin")

# 모델 로드
# loaded_model = FastText.load("fasttext_model.bin")

In [55]:
model.vector_size

100

사용 편의성을 위해 단어 삽입이 포함된 FastText 파일을 word2vec 형식으로 변환

In [56]:
from gensim.models import KeyedVectors

# FastText 모델 로드
fasttext_model = FastText.load("fasttext_model.bin")

# Word2Vec 형식으로 변환
word2vec_model = KeyedVectors(vector_size=fasttext_model.vector_size)
word2vec_model.key_to_index  = fasttext_model.wv.key_to_index
word2vec_model.vectors = fasttext_model.wv.vectors

# Word2Vec 모델 저장
# word2vec_model.save_word2vec_format("word2vec_model.bin")

# 텍스트 분류 작업에 단어 임베딩 적용
- 이제 우리는 어휘의 모든 단어에 대한 벡터인 단어 표현을 갖게 되었음
- 하지만 전체 문장을 처리해야 하므로 문장 임베딩을 만들어야 함
- 전체 문장을 나타내는 벡터가 필요하며 벡터의 모든 기능은 단어 임베딩을 기반으로 함
- 문장 임베딩의 i번째 값은 문장의 모든 단어에 대한 단어 임베딩의 i번째 값의 평균이 됨
  - "i번째 값"을 계산하기 위해 문장에 있는 모든 단어의 임베딩 벡터에서 i번째 값들을 추출하고, 이를 평균내면 됨
  - ;이 과정을 모든 차원에 대해 반복하면, 문장 임베딩이 생성됩니다.

In [57]:
class Word2VecVectorizer:
  def __init__(self, model):
    print("Loading in word vectors...")
    self.word_vectors = model
    print("Finished loading in word vectors")

  def fit(self, data):
    pass

  def transform(self, data):
    # determine the dimensionality of vectors
    v = self.word_vectors.get_vector('king')
    self.D = v.shape[0]

    X = np.zeros((len(data), self.D))
    n = 0
    test_m = 0
    emptycount = 0
    for sentence in data:
      tokens = sentence.split()
      vecs = []
      m = 0
      for word in tokens:
        try:
          # throws KeyError if word not found
          vec = self.word_vectors.get_vector(word)
          vecs.append(vec)
          m += 1
        except KeyError:
          # pass
          test_m += 1
      if len(vecs) > 0:
        vecs = np.array(vecs)
        X[n] = vecs.mean(axis=0)
      else:
        emptycount += 1
      n += 1
    print("Numer of samples with no words found: %s / %s" % (emptycount, len(data)))
    print(f"no words : {test_m}")
    return X


  def fit_transform(self, data):
    self.fit(data)
    return self.transform(data)

In [58]:
# Set a word vectorizer
vectorizer = Word2VecVectorizer(word2vec_model)

Loading in word vectors...
Finished loading in word vectors


In [59]:
# Get the sentence embeddings for the train dataset
Xtrain = vectorizer.fit_transform(X_train)
# Ytrain = train_labels
# Get the sentence embeddings for the test dataset
Xtest = vectorizer.transform(X_test)
# Ytest = test_labels

Numer of samples with no words found: 0 / 25000
no words : 36427
Numer of samples with no words found: 0 / 25000
no words : 35988


In [60]:
print(Xtrain.shape,Xtest.shape)

(25000, 100) (25000, 100)


# 문장 임베딩에 대한 분류기 훈련

## 랜덤 포레스트 분류기

In [61]:
from sklearn.ensemble import RandomForestClassifier

# create the model, train it, print scores
# forest = RandomForestClassifier(n_estimators=200)
forest = RandomForestClassifier(
    n_estimators = 100, n_jobs = -1, random_state=2018)
forest = forest.fit(Xtrain, Ytrain)

print("train score:", forest.score(Xtrain, Ytrain))
# print("test score:", clf.score(Xtest, Ytest))

train score: 1.0


In [62]:
# 테스트 데이터를 넣고 예측한다.
result = forest.predict(Xtest)
result[:10]

array([1, 0, 1, 0, 1, 1, 0, 0, 0, 1])

# 캐글 제출을 위해 예측결과 저장

In [63]:
# 예측 결과를 저장하기 위해 데이터프레임에 담아 준다.
output = pd.DataFrame(data={'id':test_path['id'], 'sentiment':result})
output.head()

,id,sentiment
0,"""12311_10""",1
1,"""8348_2""",0
2,"""5828_4""",1
3,"""7186_2""",0
4,"""12128_7""",1


In [64]:
output.to_csv('FastText.csv', index=False, quoting=3)

In [65]:
output_sentiment = output['sentiment'].value_counts()
print(np.abs(output_sentiment[0] - output_sentiment[1]))
output_sentiment

592


1    12796
0    12204
Name: sentiment, dtype: int64